In [1]:
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from os import environ


# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://api.github.com/graphql", 
                             headers={'Authorization': f"Bearer {environ['GITHUB_TOKEN']}"})

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

In [24]:

# Provide a GraphQL query
query = gql(
    """
        query($project_id: ID!, $item_number: Int!, $after: String!){
        node(id: $project_id){ 
            ... on ProjectV2 {
            items(first: $item_number, after: $after) {
                nodes{ 
                id, 
                type,
                fieldValues(first: 100){
                    nodes{
                    ... on ProjectV2ItemFieldTextValue {
                        text
                        field {
                        ... on ProjectV2FieldCommon {
                            name
                        }
                        }
                    }
                    ... on ProjectV2ItemFieldSingleSelectValue {
                        name
                        field {
                        ... on ProjectV2FieldCommon {
                            name
                        }
                        }
                    }
                    ... on ProjectV2ItemFieldIterationValue {
                        title
                        field {
                        ... on ProjectV2FieldCommon {
                            name
                        }
                        }
                    }
                    ... on ProjectV2ItemFieldNumberValue {
                        number,
                        field {
                        ... on ProjectV2FieldCommon {
                            name
                        }
                        }
                    }
                    }
                },
                content{
                    ... on DraftIssue {
                    assignees(first: 10) {
                        nodes{
                        login
                        }
                    }
                    }
                    ...on Issue {
                    assignees(first: 10) {
                        nodes{
                        login
                        }
                    }
                    }
                    ...on PullRequest {
                    assignees(first: 10) {
                        nodes{
                        login
                        }
                    }
                    }
                },
                },
                totalCount,
                pageInfo {
                endCursor
                startCursor
                hasNextPage
                hasPreviousPage
                }
            }
            }
        }
        }
"""
)

In [25]:
params = {"project_id": environ['PROJECT_ID'], "item_number":3, "after":"Mw"}
# Execute the query on the transport
# result = client.execute(query)
result = await client.execute_async(query, variable_values=params)

In [20]:
result['node']['items']['pageInfo']

{'endCursor': 'Ng',
 'startCursor': 'NA',
 'hasNextPage': True,
 'hasPreviousPage': True}

In [22]:
class Issue:
    id:str
    type:str = "DRAFT_ISSUE"
    title:str = ""
    status:str = "None"
    iteration:str = ""
    estimate:float = 0.0
    actual:float = 0.0
    assignees:list[str] = []

    def __init__(self, node:dict):
        self.id = node['id']
        self.type = node['type']
        for field in node['fieldValues']['nodes']:
            if("field" in field):
                name = field["field"]["name"]
                if(name == "Title"): self.title = field['text']
                if(name == "Iteration"): self.iteration = field['title']
                if(name == "Status"): self.status = field['name']
                if(name == "Estimate"): self.estimate = field['number']
                if(name == "Actual"): self.actual = field['number']

        for assignee in node['content']['assignees']['nodes']:
            self.assignees.append(assignee['login'])
    
    def to_dict(self):
        obj = dict({})
        obj['id'] = self.id
        obj['type'] = self.type
        obj['title'] = self.title
        obj['status'] = self.status
        obj['iteration'] = self.iteration
        obj['estimate'] = self.estimate
        obj['actual'] = self.actual
        obj['assignees'] = self.assignees
        return obj

In [28]:
issues = []
hasNextPage = True
endCursor = "null"

while hasNextPage:
    params = {"project_id": environ['PROJECT_ID'], "item_number":5, "after":endCursor}
    # Execute the query on the transport
    # result = client.execute(query)
    result = await client.execute_async(query, variable_values=params)
    for node in result['node']['items']['nodes']:
        issue = Issue(node)
        issues.append(issue.to_dict())
    # {'endCursor': 'Ng',
    #  'startCursor': 'NA',
    #  'hasNextPage': True,
    #  'hasPreviousPage': True}
    pageInfo = result['node']['items']['pageInfo']
    endCursor = pageInfo['endCursor']
    startCursor = pageInfo['startCursor']
    hasNextPage = pageInfo['hasNextPage']
    hasPreviousPage = pageInfo['hasPreviousPage']
    print(startCursor,endCursor,hasNextPage,hasPreviousPage)


NQ MQ True False
MTA Ng True True
MTU MTE True True
MjA MTY True True
MjU MjE True True
MzA MjY True True
MzU MzE True True
NDA MzY True True
NDU NDE True True
NTA NDY True True
NTU NTE True True
NjA NTY False True


In [29]:
import pandas as pd
df = pd.DataFrame.from_records(issues, index="id")

In [9]:
from datetime import datetime
time = datetime.now().strftime("%Y%m%d-%H%M%S")
time

'20240220-124752'

In [10]:
df.to_json(path_or_buf=f"/src/data/{time}.json")

In [11]:
df.to_records()

rec.array([('PVTI_lADOCU5NV84AcKb4zgMc_HY', 'DRAFT_ISSUE', 'Fixing/Recheck/Revisit Architecture That Supports the Models', 'None', 'Iteration 2', 0., 0., list(['akraradets', 'anuj-guptaa', 'guntsvzz'])),
           ('PVTI_lADOCU5NV84AcKb4zgMelz8', 'ISSUE', 'Create fields for flow account form and DB schema', 'In progress', 'Iteration 2', 0., 0., list(['akraradets', 'anuj-guptaa', 'guntsvzz'])),
           ('PVTI_lADOCU5NV84AcKb4zgMtovU', 'DRAFT_ISSUE', 'Documentation - Model APIs - Text, Image, Table/Graph', 'Review', 'Iteration 2', 0., 0., list(['akraradets', 'anuj-guptaa', 'guntsvzz']))],
          dtype=[('id', 'O'), ('type', 'O'), ('title', 'O'), ('status', 'O'), ('iteration', 'O'), ('estimate', '<f8'), ('actual', '<f8'), ('assignees', 'O')])

In [15]:
f = open("./request_string.graphql", "r").read()
f

'query($project_id: ID!){\n  node(id: $project_id){ \n    ... on ProjectV2 {\n      items(first: 3) {\n        nodes{ \n          id, \n          type,\n          fieldValues(first: 100){\n            nodes{\n              ... on ProjectV2ItemFieldTextValue {\n                text\n                field {\n                  ... on ProjectV2FieldCommon {\n                    name\n                  }\n                }\n              }\n              ... on ProjectV2ItemFieldSingleSelectValue {\n                name\n                field {\n                  ... on ProjectV2FieldCommon {\n                    name\n                  }\n                }\n              }\n              ... on ProjectV2ItemFieldIterationValue {\n                title\n                field {\n                  ... on ProjectV2FieldCommon {\n                    name\n                  }\n                }\n              }\n              ... on ProjectV2ItemFieldNumberValue {\n                number,\n     